### A question-answering demo using Astra DB and LangChain, powered by Vector Search

#### Pre-requisites:

**_Serverless Cassandra with Vector Search_** database on [Astra DB](https://astra.datastax.com) to run this demo. As outlined in more detail [here](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html#_prepare_for_using_your_vector_database), the DB Token with role _Database Administrator_ and copy Database ID: these connection parameters are needed momentarily.

#### What you will do:

- Setup: import dependencies, provide secrets, create the LangChain vector store;
- Run a Question-Answering loop retrieving the relevant headlines and having an LLM construct the answer.

Import the packages

In [ ]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from PyPDF2 import PdfReader
from langchain_community.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

: 

Key Setup

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:DTTQOMiQKzsmpGQHvyaEoKth:e397dd9ad202ab01c589ca0212ed7b1e7de0f76bddeef0787423406734d9624f" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "3b67d6de-ff55-4e11-ad7b-d4f3762e2ad7" # enter your Database ID

Input pdf file

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('data\\budget_speech.pdf')
print(pdfreader)

Extract all text from PDF

In [ ]:
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content
print(raw_text)

Initialize the connection to your database:

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
# Local LLM via Ollama
llm = Ollama(model="llama3.2")

# Local embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding

Create your LangChain vector store ... backed by Astra DB!

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="pdf_qa_query_db",
    session=None,
    keyspace=None,
)
astra_vector_store

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)
len(texts)

In [ ]:
texts[:5]

Load the dataset into the vector store

In [ ]:
astra_vector_store.add_texts(texts)

print("Inserted %i headlines." % len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

### Run the QA cycle

Simply run the cells and ask a question -- or `quit` to stop. (you can also stop execution with the "▪" button on the top toolbar)

Here are some suggested questions:
- _What is the current GDP?_
- _How much the agriculture target will be increased to and what the focus will be_


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))